In [36]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras

In [37]:
from tensorflow.keras.datasets import mnist

In [72]:
fashion_mnist=keras.datasets.fashion_mnist
(x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()

In [73]:
x_train.shape

(60000, 28, 28)

In [74]:
#一個channel，所以(28,28)-->(28,28,1)

In [75]:
x_train=x_train.reshape(60000,28,28,1)/255 #正規化

In [76]:
x_test=x_test.reshape(10000,28,28,1)/255

In [77]:
x_train[87].shape

(28, 28, 1)

In [78]:
y_train[87]

7

In [79]:
from tensorflow.keras.utils import to_categorical

In [80]:
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

In [81]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

In [82]:
#step1 打造函數學習機(CNN)

In [83]:
model=Sequential()

In [84]:
model.add(Conv2D(16,(3,3),padding='same',input_shape=(28,28,1),activation='relu')) #filter,矩陣3*3

In [85]:
#輸出16個28x28矩陣
#事實上是(28,28,16)

In [86]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [87]:
#(14,14,16)

In [88]:
model.add(Conv2D(32,(3,3),padding='same',activation='relu'))

In [89]:
#output(14,14,32)

In [90]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [91]:
#output(7,7,32)

In [92]:
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))

In [93]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [94]:
model.add(Flatten())

In [95]:
model.add(Dense(10,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(32,activation='relu'))

In [96]:
model.add(Dense(10,activation='softmax'))

In [97]:
#看一下神經網路

In [98]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
__________

In [99]:
#conv2d (Conv2D):
#3*3(權重)+1(bias)
(3*3+1)*16

160

In [100]:
model.compile(loss='mse',optimizer=Adam(lr=0.003),metrics=['accuracy'])

In [101]:
#step2 fit

In [102]:
model.fit(x_train,y_train,batch_size=100,epochs=15)

Epoch 1/15
60000/60000 [==============================] - 23s 376us/sample - loss: 0.0314 - acc: 0.7688
Epoch 2/15
60000/60000 [==============================] - 23s 388us/sample - loss: 0.0192 - acc: 0.8673
Epoch 3/15
60000/60000 [==============================] - 24s 397us/sample - loss: 0.0167 - acc: 0.8848
Epoch 4/15
60000/60000 [==============================] - 22s 373us/sample - loss: 0.0151 - acc: 0.8965
Epoch 5/15
60000/60000 [==============================] - 22s 369us/sample - loss: 0.0141 - acc: 0.9046
Epoch 6/15
60000/60000 [==============================] - 22s 367us/sample - loss: 0.0133 - acc: 0.9101
Epoch 7/15
60000/60000 [==============================] - 23s 388us/sample - loss: 0.0125 - acc: 0.9164
Epoch 8/15
60000/60000 [==============================] - 23s 389us/sample - loss: 0.0120 - acc: 0.9190
Epoch 9/15
60000/60000 [==============================] - 23s 384us/sample - loss: 0.0113 - acc: 0.9248
Epoch 10/15
60000/60000 [==============================] - 22s 3

In [103]:
score = model.evaluate(x_test, y_test)
result=model.predict_classes(x_test)

10000/10000 [==============================] - 2s 189us/sample - loss: 0.0135 - acc: 0.9100


In [104]:
score = model.evaluate(x_train, y_train)
result=model.predict_classes(x_train)

60000/60000 [==============================] - 10s 163us/sample - loss: 0.0080 - acc: 0.9488


In [105]:
from ipywidgets import interact_manual

In [109]:
#step3 預測
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
result=model.predict_classes(x_test)
def my_predict(n):
    for i in range(0,9):
        if y_test[n][i]!=0:
            a=i
    print('CNN預測是：',class_names[result[n]] ,', 正確答案是：',class_names[a])
    X=x_test[n].reshape(28,28)
    plt.imshow(X,cmap='Greys')
interact_manual(my_predict,n=(0,9999))

A Jupyter Widget

<function __main__.my_predict>

In [110]:
loss,acc=score

In [111]:
print('正確率為：',acc)

正確率為： 0.9488


In [104]:
#改變的地方：Dense層數、學習率、optimizer-->Adam、batch_size、學習回合數
#使用SGD試過很多次，但準確率都不到80%，optimizer改成Adam之後準確率變高，Dense變多層，用來提高準確率，
#最後測試的準確率比訓練的準確率低了一些